In [ ]:
!pip install scikit-plot -q

In [21]:
import openpyxl #biblioteca para abrir arquivos em excel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV # biblioteca para treino da arvore de decisão e otimização de hiperparâmetros
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor #para classificação e regressão as variáveis
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix # biblioteca para métricas do modelo
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier # biblioteca de rede neural
#bibliotecas para visualização das árvore arvore de decisão
from sklearn.tree import export_text
from sklearn import tree
from IPython.display import Image  
from io import StringIO 
from sklearn.tree import export_graphviz
import pydot 
from sklearn.datasets import load_iris
import graphviz

In [8]:
#comando para exibição de todas as colunas do meio do dataframe que, este este, seriam ocultadas pelo jupyter.
pd.set_option('display.max_columns', 25)

In [9]:
#carregando os dados
bd1 = pd.read_excel("defaultCreditcardClients.xls")

In [ ]:
# Aqui retiramos a linha onde tinha apenas o nome da variável, pois esta estava sendo contada como valor sem o ser.
#Logo, bd1_copy é o mesmo bd1 só que sem os cabeçalhos descritivos
bd1_copy = bd1.iloc[1:]
bd1_copy

In [14]:
#exclusão da coluna 'Unnamed: 0'
bd1.drop('Unnamed: 0', axis=1, inplace=True)

In [15]:
# Dividindo os dados em features eixo(X) e variável alvo eixo(y)
X = bd1_copy.drop(['Y'],  axis=1)
y = bd1_copy['Y'].astype(int)

In [16]:
# Dividindo os dados em features eixo(X) e variável alvo eixo(y)
X = bd1_copy.drop(columns=['Y'],  axis=1)
y = bd1_copy['Y'].astype(int)

In [18]:
# Divida os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [33]:
# Defina modelos individuais (árvore de decisão e rede neural, por exemplo)
tree_model = RandomForestClassifier(n_estimators=1, random_state=42)
neural_network_model = MLPClassifier(hidden_layer_sizes=(50, 50), max_iter=500, random_state=42)




In [34]:
# Crie o ensemble com VotingClassifier
ensemble_model = VotingClassifier(estimators=[
    ('tree', tree_model),
    ('neural_network', neural_network_model)
], voting='hard')  # 'hard' para votação por maioria, 'soft' para votação ponderada



In [35]:
# Treine o modelo ensemble
ensemble_model.fit(X_train, y_train)



VotingClassifier(estimators=[('tree',
                              RandomForestClassifier(n_estimators=1,
                                                     random_state=42)),
                             ('neural_network',
                              MLPClassifier(hidden_layer_sizes=(50, 50),
                                            max_iter=500, random_state=42))])

In [36]:
# Faça previsões no conjunto de teste
y_pred = ensemble_model.predict(X_test)



In [37]:
# Avalie o modelo ensemble
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia no conjunto de teste:", accuracy)

Acurácia no conjunto de teste: 0.7838333333333334
